In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.tabular.all import *
from wwf.tab.export import *
from bnbsl import *

from classes import *
from utils import *

In [3]:
SEASON = '21'

## Download HKJC odds

In [4]:
path_raw = Path('raw_data')
path_data = Path('data')

path_output = Path('output')

In [5]:
session = requests.Session()
r = session.get('http://bet.hkjc.com')
cookies = r.cookies

In [6]:
odds_url = 'https://bet.hkjc.com/football/getJSON.aspx?jsontype=odds_chl.aspx'
response = session.post(
    odds_url,
    headers={'referer':'http://bet.hkjc.com'},
    cookies=cookies
)

In [7]:
with open(path_data/'json'/f'odds_chl-{datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.txt', 'w') as f:
    f.write(response.text)

In [8]:
matches_json = json.loads(response.text)[1]['matches']

In [9]:
# Example
# matches_json = json.loads(open(path_data/'json'/'odds_chl-2021-09-21-14-03-17.txt').read())[1]['matches']

In [10]:
matches = [Match(m) for m in matches_json if m['matchStatus'] == 'Defined']
odds = [m.export() for m in matches]

['20211106SAT54', '2021-Nov-07', '00:00:00', 'French Division 1 [FFL]', 'Lille', 'Angers SCO', 'false', '12.5', '5.10', '1.12', 'false', '10.5', '2.80', '1.38', 'true', '9.5', '2.15', '1.62']
['20211106SAT55', '2021-Nov-07', '00:00:00', 'Russian Premier [RPL]', 'PFC Sochi', 'CSKA Moscow', 'false', '10.5', '2.43', '1.49', 'false', '13.5', '5.10', '1.12', 'true', '9.5', '1.90', '1.80']
['20211106SAT58', '2021-Nov-07', '01:00:00', 'Italian Division 1 [ISA]', 'Juventus', 'Fiorentina', 'true', '10.5', '2.29', '1.55', 'false', '13.5', '5.10', '1.12', 'false', '11.5', '2.95', '1.34']
['20211106SAT62', '2021-Nov-07', '01:30:00', 'Eng Premier [EPL]', 'Brighton', 'Newcastle', 'false', '14.5', '5.80', '1.09', 'true', '10.5', '2.00', '1.72', 'false', '11.5', '2.55', '1.45']
['20211106SAT63', '2021-Nov-07', '01:30:00', 'Spanish Division 1 [SFL]', 'Alaves', 'Levante', 'false', '13.5', '5.30', '1.11', 'true', '9.5', '1.88', '1.82', 'false', '10.5', '2.40', '1.50']
['20211106SAT64', '2021-Nov-07', '01

In [11]:
cols_match = ['MatchDay', 'Date', 'Time', 'LeagueJC', 'HomeTeamJC', 'AwayTeamJC']
cols_odds0 = ['MAINLINE_0', 'CHL_LINE_0', 'CHL_H_0', 'CHL_L_0']
cols_odds1 = ['MAINLINE_1', 'CHL_LINE_1', 'CHL_H_1', 'CHL_L_1']
cols_odds2 = ['MAINLINE_2', 'CHL_LINE_2', 'CHL_H_2', 'CHL_L_2']
cols_odds  = ['MAINLINE', 'CHL_LINE', 'CHL_H', 'CHL_L']
cols_pred  = ['alpha_1', 'alpha_2', 'mu_1', 'mu_2', 'omega', 'corr']

cols = cols_match + cols_odds0 + cols_odds1 + cols_odds2

In [12]:
odds = pd.DataFrame(odds, columns=cols)
odds = odds.fillna(value=np.nan)

In [13]:
cols_odds_ = cols_odds0[1:]+cols_odds1[1:]+cols_odds2[1:]
odds[cols_odds_] = odds[cols_odds_].astype(float)

In [14]:
odds['MatchDay'] = odds['MatchDay'].str[8:]
odds['Date'] = pd.to_datetime(odds['Date'])
odds['Time'] = pd.to_datetime(odds['Time'], format='%H:%M:%S').dt.time

In [15]:
odds['DateTimeJC'] = pd.to_datetime(odds['Date'].dt.date.map(str) + '-' + odds['Time'].map(str))
odds['DateTimeJC'] = odds['DateTimeJC'].dt.tz_localize('Hongkong')
odds['DateTime'] = odds['DateTimeJC'].dt.tz_convert('GB')

In [16]:
map_league = pd.read_csv(path_data/'league.csv')
map_team = pd.read_csv(path_data/'team.csv')

In [17]:
# Map Div name
odds = odds.merge(map_league[['LeagueJC', 'Div']], 'inner', on='LeagueJC')

In [18]:
# Map Team name
odds = odds.merge(map_team[['TeamNameJC', 'TeamName']].rename(columns={'TeamName':'HomeTeam'}), 'inner', 
                  left_on='HomeTeamJC', right_on='TeamNameJC').drop(columns=['TeamNameJC'])
odds = odds.merge(map_team[['TeamNameJC', 'TeamName']].rename(columns={'TeamName':'AwayTeam'}), 'inner', 
                  left_on='AwayTeamJC', right_on='TeamNameJC').drop(columns=['TeamNameJC'])

## Download recent stats

In [20]:
# Download latest results in current season
!wget -q https://www.football-data.co.uk/mmz4281/{SEASON}{int(SEASON)+1}/data.zip -O raw_data/data.zip

# Unzip to folder
!unzip -q -o raw_data/data.zip -d raw_data/{SEASON}

In [19]:
usecols = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'HC', 'AC', 'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST']
dtype = {'HC':'float', 'AC':'float'}
parse_dates = ['Date']

seasons = [SEASON]

dfs = []

for folder in sorted(path_raw.iterdir()):
    if folder.is_dir() and folder.name in seasons: 
        for file in sorted(folder.glob('*.csv')):
            try:
                df = pd.read_csv(file, usecols=usecols, dtype=dtype, parse_dates=parse_dates, dayfirst=True)
                df['Season'] = folder.name
                dfs.append(df)
            except:
                continue

In [20]:
df_season = pd.concat(dfs)
df_season = df_season.dropna()
df_season = df_season.sort_values(['Div', 'Date', 'HomeTeam']).reset_index(drop=True)

In [21]:
df_hist = pd.read_csv(path_data/'data.csv', dtype={'HC':'float', 'AC':'float'}, parse_dates=['Date'])
df_hist = df_hist.query(f'Season == {int(SEASON)-1}').reset_index(drop=True)

In [22]:
df_season = pd.concat([df_hist[df_season.columns], df_season])

In [23]:
# Make features on historical stats (Home and Away)
stats = ['FTHG', 'HS', 'HST', 'HC', 'FTAG', 'AS', 'AST', 'AC']
df_home, df_away = joinLastGamesStatsHomeAway(df_season, stats)

In [24]:
# Make features on historical stats (For and Against)
stats = [('FTHG', 'FTAG', 'FTG'), ('HS', 'AS', 'S'), ('HST', 'AST', 'ST'), ('HC', 'AC', 'C')]
df_for, df_against = joinLastGamesStatsForAgainst(df_season, stats)

In [25]:
df_home = df_home.sort_values(['HomeTeam', 'Date']).reset_index(drop=True)
df_away = df_away.sort_values(['AwayTeam', 'Date']).reset_index(drop=True)

df_home = df_home.groupby('HomeTeam')[df_home.columns[df_home.columns.str.contains('Avg')]].last().reset_index()
df_away = df_away.groupby('AwayTeam')[df_away.columns[df_away.columns.str.contains('Avg')]].last().reset_index()

In [26]:
odds = odds.merge(df_home, 'left', 'HomeTeam').merge(df_away, 'left', 'AwayTeam')

In [27]:
cols_home = df_for.columns[df_for.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_for.columns[df_for.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df_for = df_for.groupby('Team')[df_for.columns[df_for.columns.str.contains('Avg')]].last().reset_index()
odds = odds.merge(df_for.rename(columns=cols_home), 'left', 'HomeTeam').merge(df_for.rename(columns=cols_away), 'left', 'AwayTeam')

In [28]:
cols_home = df_against.columns[df_against.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_against.columns[df_against.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df_against = df_against.groupby('Team')[df_against.columns[df_against.columns.str.contains('Avg')]].last().reset_index()
odds = odds.merge(df_against.rename(columns=cols_home), 'left', 'HomeTeam').merge(df_against.rename(columns=cols_away), 'left', 'AwayTeam')

In [29]:
add_datepart(odds, 'DateTime', prefix='', drop=False);

In [30]:
display_df(odds.head(5).T)

,0,1,2,3,4
MatchDay,SAT54,SAT85,SUN26,SUN43,SUN44
Date,2021-11-07 00:00:00,2021-11-07 00:00:00,2021-11-07 00:00:00,2021-11-07 00:00:00,2021-11-07 00:00:00
Time,00:00:00,04:00:00,20:00:00,22:00:00,22:00:00
LeagueJC,French Division 1 [FFL],French Division 1 [FFL],French Division 1 [FFL],French Division 1 [FFL],French Division 1 [FFL]
HomeTeamJC,Lille,Bordeaux,Marseille,Lorient,Nantes
AwayTeamJC,Angers SCO,Paris Saint Germain,Metz,Brest,Strasbourg
MAINLINE_0,false,true,false,true,false
CHL_LINE_0,12.5,9.5,13.5,9.5,10.5
CHL_H_0,5.1,1.75,5.1,1.82,2.3
CHL_L_0,1.12,1.95,1.12,1.88,1.54


## Load model

In [31]:
learn_bnb = load_learner('models/learn_bnbsl.pkl')

In [32]:
to = load_pandas('models/to.pkl')

In [33]:
def predict(self, row):
    "Predict on a Pandas Series"
    dl = self.dls.test_dl(row.to_frame().T)
    dl.dataset.conts = dl.dataset.conts.astype(np.float32)
    inp,preds,_ = self.get_preds(dl=dl, with_input=True, with_decoded=False)
    b = tuplify(inp)
    full_dec = self.dls.decode(b)
    return full_dec,preds[0]

learn_bnb.predict = MethodType(predict, learn_bnb)

In [34]:
to_tst = to.new(odds)
to_tst.process()
# to_tst.items.head()

In [35]:
tst_dl = learn_bnb.dls.valid.new(to_tst)
tst_dl.show(max_n=999)

,Div,HomeTeam,AwayTeam,Dayofweek,FTHGLast5Avg,HSLast5Avg,HSTLast5Avg,HCLast5Avg,FTAGLast5Avg,ASLast5Avg,ASTLast5Avg,ACLast5Avg,HomeFTGForLast5Avg,HomeSForLast5Avg,HomeSTForLast5Avg,HomeCForLast5Avg,AwayFTGForLast5Avg,AwaySForLast5Avg,AwaySTForLast5Avg,AwayCForLast5Avg,HomeFTGAgainstLast5Avg,HomeSAgainstLast5Avg,HomeSTAgainstLast5Avg,HomeCAgainstLast5Avg,AwayFTGAgainstLast5Avg,AwaySAgainstLast5Avg,AwaySTAgainstLast5Avg,AwayCAgainstLast5Avg,Year,Month,Week,Day,Dayofyear
0,F1,Lille,Angers,5,1.2,11.8,3.6,4.2,1.2,9.4,3.0,4.8,1.2,10.0,4.0,5.4,1.2,12.2,3.8,8.0,0.8,10.6,3.6,5.4,1.8,7.0,2.4,3.4,2021.0,11.0,44.0,6.0,310.0
1,F1,Bordeaux,Paris SG,5,1.4,11.2,4.2,3.8,2.0,13.6,3.8,4.0,1.4,10.0,3.0,4.0,1.6,14.2,3.4,6.2,1.8,14.0,6.0,6.8,1.0,9.4,3.4,4.6,2021.0,11.0,44.0,6.0,310.0
2,F1,Marseille,Metz,6,2.6,15.2,4.6,5.0,1.2,9.0,4.0,4.2,1.2,12.2,3.0,4.4,1.0,7.6,3.0,3.8,1.2,12.0,4.4,4.6,2.4,15.0,5.2,6.4,2021.0,11.0,44.0,7.0,311.0
3,F1,Lorient,Brest,6,1.4,7.2,2.6,2.8,1.0,9.0,3.6,4.8,0.8,9.4,3.4,4.6,1.0,8.2,2.2,4.4,1.2,11.8,3.0,5.8,1.8,12.0,5.0,4.6,2021.0,11.0,44.0,7.0,311.0
4,F1,Nantes,Strasbourg,6,1.4,14.0,5.8,5.4,1.4,8.6,3.0,4.2,2.2,10.8,4.6,4.8,2.2,8.8,3.8,5.0,1.2,10.4,3.4,5.2,0.8,12.8,4.4,5.8,2021.0,11.0,44.0,7.0,311.0
5,F1,Reims,Monaco,6,1.4,11.8,4.6,4.8,1.4,10.4,3.2,3.8,1.0,9.2,2.8,4.4,2.2,9.8,3.8,4.6,1.2,9.2,4.0,5.8,1.2,10.2,3.8,3.4,2021.0,11.0,44.0,7.0,311.0
6,F1,St Etienne,Clermont,6,0.8,14.2,4.4,5.6,1.2,12.4,4.4,6.6,0.8,10.8,4.4,4.4,0.8,8.0,3.2,5.0,2.8,13.6,5.8,5.6,2.2,9.4,3.4,4.8,2021.0,11.0,44.0,7.0,311.0
7,F1,Nice,Montpellier,6,2.2,14.4,5.2,4.2,1.4,9.0,3.8,5.8,1.6,15.0,5.8,5.2,1.2,11.6,4.4,5.0,1.0,9.2,4.0,3.0,1.4,12.8,4.6,4.2,2021.0,11.0,44.0,7.0,311.0
8,F1,Rennes,Lyon,6,2.0,13.2,4.4,4.4,1.6,13.6,6.2,4.8,2.4,17.0,6.4,6.4,1.6,13.6,5.6,5.2,0.6,10.6,2.4,5.8,1.0,13.0,5.2,5.2,2021.0,11.0,44.0,7.0,311.0
9,I1,Juventus,Fiorentina,5,1.6,12.8,5.0,4.0,1.2,10.4,3.8,4.8,1.8,14.8,4.8,5.4,1.2,12.6,3.6,4.4,1.0,10.2,3.2,4.4,1.2,9.6,3.8,3.4,2021.0,11.0,44.0,6.0,310.0


In [36]:
pred, _ = learn_bnb.get_preds(dl=tst_dl)

In [37]:
alpha = F.softplus(pred[:, 0:2])
mu = F.softplus(pred[:, 2:4])
omega = torch.tanh(pred[:, 4]) * 0.0

total_count = 1. / alpha
logits = torch.log(alpha * mu)

# print(alpha, mu, omega)

In [38]:
d = 1.-math.exp(-1)
corr = omega * d**2 * torch.sqrt(mu.prod(-1)*(1.+alpha*mu).prod(-1)) * ((1.+d*alpha*mu)**(-1-1/alpha)).prod(-1)

In [39]:
odds[cols_pred] = torch.cat([alpha, mu, omega.unsqueeze(-1), corr.unsqueeze(-1)], dim=-1)

In [40]:
odds0 = odds[cols_match+cols_odds0+cols_pred].rename(columns=dict(zip(cols_odds0, cols_odds)))
odds1 = odds[cols_match+cols_odds1+cols_pred].rename(columns=dict(zip(cols_odds1, cols_odds)))
odds2 = odds[cols_match+cols_odds2+cols_pred].rename(columns=dict(zip(cols_odds2, cols_odds)))

In [41]:
odds = pd.concat([odds0, odds1, odds2]).dropna().reset_index(drop=True)
odds['MAINLINE'] = np.where(odds['MAINLINE']=='true', True, False)

In [42]:
odds.head(10)

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,alpha_1,alpha_2,mu_1,mu_2,omega,corr
0,SAT54,2021-11-07,00:00:00,French Division 1 [FFL],Lille,Angers SCO,False,12.5,5.10,1.12,0.081081,0.070067,4.686617,4.046116,-0.0,-0.0
1,SAT85,2021-11-07,04:00:00,French Division 1 [FFL],Bordeaux,Paris Saint Germain,True,9.5,1.75,1.95,0.091823,0.033468,3.730953,6.885892,0.0,0.0
2,SUN26,2021-11-07,20:00:00,French Division 1 [FFL],Marseille,Metz,False,13.5,5.10,1.12,0.034174,0.075592,6.731087,3.545680,-0.0,-0.0
3,SUN43,2021-11-07,22:00:00,French Division 1 [FFL],Lorient,Brest,True,9.5,1.82,1.88,0.086024,0.065711,4.577596,4.497922,0.0,0.0
4,SUN44,2021-11-07,22:00:00,French Division 1 [FFL],Nantes,Strasbourg,False,10.5,2.30,1.54,0.046139,0.086134,6.425599,3.829643,-0.0,-0.0
5,SUN45,2021-11-07,22:00:00,French Division 1 [FFL],Reims,Monaco,False,13.5,5.30,1.11,0.159502,0.123976,3.379888,3.989213,0.0,0.0
6,SUN46,2021-11-07,22:00:00,French Division 1 [FFL],St. Etienne,Clermont,False,13.5,5.10,1.12,0.105532,0.088604,4.305065,4.136442,-0.0,-0.0
7,SUN59,2021-11-08,00:00:00,French Division 1 [FFL],Nice,Montpellier,False,13.5,4.90,1.13,0.063020,0.074286,5.178190,3.912096,-0.0,-0.0
8,SUN89,2021-11-08,03:45:00,French Division 1 [FFL],Rennes,Lyon,False,12.5,4.15,1.18,0.055905,0.076154,5.672256,4.174709,-0.0,-0.0
9,SAT58,2021-11-07,01:00:00,Italian Division 1 [ISA],Juventus,Fiorentina,True,10.5,2.29,1.55,0.046071,0.058636,6.132838,4.810640,-0.0,-0.0


In [43]:
prob_hilo = []

for r in list(zip(odds['alpha_1'], odds['alpha_2'], odds['mu_1'], odds['mu_2'], odds['omega'], odds['CHL_LINE'])):
    total_count = 1. / torch.tensor(r[0:2], device='cpu') 
    logits = torch.log(torch.tensor(r[0:2], device='cpu') * torch.tensor(r[2:4], device='cpu'))
    omega = torch.tensor(r[4], device='cpu')

    bnb_corner = BivariateNegativeBinomialSL(total_count=total_count, omega=omega, logits=logits)
    value = torch.cartesian_prod(torch.arange(0., 15.), torch.arange(0., 15.))
    corner = bnb_corner.log_prob(value).exp()
    
    line = r[5]
    mask = value.sum(-1) < line
    prob_lo = corner[mask].sum()
    prob_hi = 1 - prob_lo
    
    prob_hilo.append([prob_hi.item(), prob_lo.item()])

In [44]:
odds[['prob_hi', 'prob_lo']] = prob_hilo

In [45]:
odds['kelly_hi'] = (odds['prob_hi'] * odds['CHL_H'] - 1) / (odds['CHL_H'] - 1)
odds['kelly_lo'] = (odds['prob_lo'] * odds['CHL_L'] - 1) / (odds['CHL_L'] - 1)

In [46]:
odds['kelly'] = np.where(
    np.maximum(odds['kelly_hi'], odds['kelly_lo']) > 0, 
    np.where(odds['kelly_hi'] > odds['kelly_lo'], odds['kelly_hi'], odds['kelly_lo']), 
    np.nan
)

In [47]:
odds['bet'] = np.where(
    np.maximum(odds['kelly_hi'], odds['kelly_lo']) > 0, 
    np.where(odds['kelly_hi'] > odds['kelly_lo'], 'High', 'Low'), 
    None
)

In [48]:
odds = odds.sort_values('kelly', ascending=False).reset_index(drop=True)

In [49]:
odds['selected'] = np.where(
    odds['MAINLINE']==True, np.where(
        odds['kelly']>0.3, '$$$', np.where(
            odds['kelly']>0.2, '$$', np.where(
                odds['kelly']>0.1, '$', None))), 
    None
)

In [50]:
odds = odds.drop(columns=cols_pred+['kelly_hi', 'kelly_lo'])

In [51]:
odds[odds.bet.notna() & odds.selected.notna()]

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,prob_hi,prob_lo,kelly,bet,selected
2,SUN45,2021-11-07,22:00:00,French Division 1 [FFL],Reims,Monaco,True,9.5,1.85,1.85,0.241744,0.758256,0.473851,Low,$$$
6,SUN75,2021-11-08,01:30:00,Spanish Division 1 [SFL],Osasuna,Real Sociedad,True,8.5,1.88,1.82,0.309398,0.690602,0.313287,Low,$$$
10,SUN67,2021-11-08,00:30:00,Eng Premier [EPL],West Ham,Liverpool,True,10.5,1.80,1.90,0.338639,0.661361,0.285095,Low,$$
11,SUN56,2021-11-07,23:15:00,Spanish Division 1 [SFL],Valencia,Atletico Madrid,True,8.5,1.73,1.98,0.355712,0.644288,0.281318,Low,$$
12,SUN33,2021-11-07,21:00:00,Spanish Division 1 [SFL],Villarreal,Getafe,True,8.5,1.85,1.85,0.334226,0.665774,0.272567,Low,$$
16,SUN46,2021-11-07,22:00:00,French Division 1 [FFL],St. Etienne,Clermont,True,9.5,1.82,1.88,0.349482,0.650518,0.253378,Low,$$
17,SUN51,2021-11-07,22:30:00,German Division 1 [GSL],Hertha Berlin,Leverkusen,True,9.5,1.83,1.87,0.348026,0.651974,0.251945,Low,$$
20,SUN88,2021-11-08,03:45:00,Italian Division 1 [ISA],AC Milan,Inter Milan,True,10.5,2.19,1.60,0.567970,0.432030,0.204919,High,$$
22,SUN30,2021-11-07,20:30:00,German Division 2 [GD2],Aue,Heidenheim,True,9.5,1.77,1.93,0.394291,0.605709,0.181740,Low,$
23,SUN90,2021-11-08,04:00:00,Spanish Division 1 [SFL],Betis,Sevilla,True,8.5,1.73,1.98,0.652506,0.347494,0.176487,High,$


In [52]:
display_df(odds[odds.MatchDay.isin(odds[odds.bet.notna() & odds.selected.notna()].MatchDay) & odds.bet.notna()])

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,prob_hi,prob_lo,kelly,bet,selected
0,SUN45,2021-11-07,22:00:00,French Division 1 [FFL],Reims,Monaco,False,13.5,5.30,1.11,0.046985,0.953015,0.525880,Low,None
1,SUN45,2021-11-07,22:00:00,French Division 1 [FFL],Reims,Monaco,False,10.5,2.30,1.54,0.168534,0.831466,0.519366,Low,None
2,SUN45,2021-11-07,22:00:00,French Division 1 [FFL],Reims,Monaco,True,9.5,1.85,1.85,0.241744,0.758256,0.473851,Low,$$$
3,SUN67,2021-11-08,00:30:00,Eng Premier [EPL],West Ham,Liverpool,False,14.5,4.75,1.14,0.076680,0.923320,0.375603,Low,None
4,SUN75,2021-11-08,01:30:00,Spanish Division 1 [SFL],Osasuna,Real Sociedad,False,9.5,2.45,1.48,0.219701,0.780299,0.322588,Low,None
5,SUN67,2021-11-08,00:30:00,Eng Premier [EPL],West Ham,Liverpool,False,11.5,2.25,1.57,0.247420,0.752580,0.318509,Low,None
6,SUN75,2021-11-08,01:30:00,Spanish Division 1 [SFL],Osasuna,Real Sociedad,True,8.5,1.88,1.82,0.309398,0.690602,0.313287,Low,$$$
7,SUN33,2021-11-07,21:00:00,Spanish Division 1 [SFL],Villarreal,Getafe,False,12.5,5.55,1.10,0.063439,0.936561,0.302175,Low,None
8,SUN56,2021-11-07,23:15:00,Spanish Division 1 [SFL],Valencia,Atletico Madrid,False,9.5,2.20,1.59,0.260625,0.739375,0.297637,Low,None
9,SUN33,2021-11-07,21:00:00,Spanish Division 1 [SFL],Villarreal,Getafe,False,9.5,2.40,1.50,0.235515,0.764485,0.293455,Low,None


In [53]:
odds.to_csv(path_output/f'odds-{datetime.now().strftime("%Y-%m-%d")}.csv', float_format='%.2f', index=False)

## END